In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

# read IRS data of 2016 
IRS2016 = pd.read_csv("/home/msari/Project1/Data/IRS_temp_2016.csv", low_memory = False)
print( "This is IRS data shape:", IRS2016.shape)
IRS2016 = IRS2016.rename(columns = {"Filer.EIN" : "EIN"})
print("The unique number of hospitals in IRS for 2016:", len(IRS2016['EIN'].value_counts()))

## read American Hospital Association (AHA) 2016 data to pandas.
AHA = pd.read_csv("/home/msari/Project1/RawData/AHA/2016/FY2016-Annual-Survey-Database/COMMA/ASPUB16.CSV", low_memory = False)
## create a subset of AHA data.
## now let's create an id data for AHA consisting of only id variables for providers
id_variables = ['ID', 'NPINUM', 'MCRNUM', 'MNAME','SYSNAME', 'SYSID', 'STCD', 'DTBEG', 'DTEND', 'FISYR', 'SERV', 'HSACODE', 'HSANAME', 'HRRNAME', 'HRRCODE', 'MLOCADDR', 'MLOCCITY', 'MLOCSTD', 'MLOCZIP' ,'SYSADDR' ,'SYSCITY' ,'SYSST' ,'SYSZIP' ,'SYSAREA' ,'SYSTELN' ,'SYSTEM_PRIMARY_CONTACT' ,'SYSTITLE' ,'LAT' ,'LONG' ,'CNTYNAME' ,'CBSANAME' ,'CBSATYPE' ,'CBSACODE' ,'DIVNAME' ,'DIVCODE' ,'CSANAME' ,'CSACODE' ,'MCNTYCD' ,'FCOUNTY' ,'FSTCD' ,'FCNTYCD', 'CNTRL']
aha2016 = AHA.loc[:, id_variables]
print( "This is AHA data shape:", aha2016.shape)
print("The unique number of hospitals in AHA for 2016:", len(aha2016['ID'].value_counts()))


This is IRS data shape: (2666, 420)
The unique number of hospitals in IRS for 2016: 2349
This is AHA data shape: (6239, 42)
The unique number of hospitals in AHA for 2016: 6239


/usr/local/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [4]:
bins = [0, 20, 30, 40, np.inf]
names = [ 'Government-NonFederal', 'non-profit', 'for-profit', 'Government-Federal']
aha2016['ControlCode'] = pd.cut(aha2016['CNTRL'], bins, labels=names)
aha2016['ControlCode'].value_counts()

non-profit               3107
for-profit               1690
Government-NonFederal    1228
Government-Federal        214
Name: ControlCode, dtype: int64

IRS data includes only non-profit hospitals or hospital systems, however, AHA data includes non-profit, for-profit and government (public) hospitals. There is no key identifier to match these two datasets. I need to use name, address, zipcode, city, and state columns to fuzzy-match those hospitals that are in both datasets. Key identifier in IRS is EIN, the key Identifier in AHA is ID variable.

I will match two datasets based on aha2016['address'] and IRS2016['address'] which are created by combining street address, city, state and zipcode columns in each dataset.

*Problem*: 

    -how to add name column to matching code?
    -how to add key identifiers to matching table so it can be merged with other variables?

In [6]:
# little manipulation for better matching (uppercase) and renaming variables to make it readable. 

# uppercase
IRS2016['Filer.USAddress.CityNm'] = IRS2016['Filer.USAddress.CityNm'].str.upper()
aha2016.SYSCITY = aha2016.SYSCITY.str.upper()
IRS2016['Filer.USAddress.AddressLine1Txt'] = IRS2016['Filer.USAddress.AddressLine1Txt'].str.upper()
aha2016['SYSADDR'] = aha2016['SYSADDR'].str.upper()

# renaming columns in IRS
IRS2016.rename(columns={'Filer.USAddress.CityNm' : 'SYSCITY', 'Filer.USAddress.StateAbbreviationCd': 'STATE', 'Filer.USAddress.AddressLine1Txt': 'ADDRESS'}, inplace=True)

## zipcode columns is not clean some are more than 5 digits.
d = []
for n in IRS2016['Filer.USAddress.ZIPCd']:
    d.append(str(n)[:5])
    
IRS2016['zipcode'] = pd.DataFrame(d)

L = []
for n in aha2016['SYSZIP']:
    L.append(str(n)[:5])
aha2016['zipcode'] = pd.DataFrame(L)

# I create a new address column that combines street address, city, state and zipcode for fuzzymatching.
aha2016['address'] = aha2016['SYSADDR'].str.cat(aha2016[['SYSCITY', 'SYSST', 'zipcode']], sep=' , ')
IRS2016['address'] =  IRS2016['ADDRESS'].str.cat(IRS2016[['SYSCITY', 'STATE', 'zipcode']], sep = ' , ')

In [7]:
## Here I am using address column that I created above to match two datasets.

def match_name(name, list_names, min_score=0):
    # -1 score incase no matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.token_sort_ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)

# List for dicts for easy dataframe creation
dict_list = []
# iterating over our addresses without any further info
for address in aha2016['address']:
    # Use the method to find best match, I can set a threshold here
    match = match_name(address, IRS2016['address'], 75)
    # New dict for storing data
    dict_ = {}
    dict_.update({"AHA_adress" : address})
    dict_.update({"IRS_address" : match[0]})
    dict_.update({"score" : match[1]})
    dict_list.append(dict_)
    
merge_table1 = pd.DataFrame(dict_list)
merge_table1

,AHA_adress,IRS_address,score
0,"5109 LEESBURG PIKE , FALLS CHURCH , VA , 22041",,-1
1,NaN,NaN,100
2,"603 MAIN STREET , WINDERMERE , FL , 34786",,-1
3,NaN,NaN,100
4,NaN,NaN,100
5,"BUILDING 'A' - MEDICAL CENTER , SAN JUAN , PR ...",,-1
6,NaN,NaN,100
7,NaN,NaN,100
8,"3660 GRANDVIEW PARKWAY, SUITE 200 , BIRMINGHAM...",,-1
9,NaN,NaN,100


In [9]:
## Matching function somehow matches NaNs and could not locate the problem. Here I am dropping those matches.
merge_table_upd = merge_table1.dropna()
merge_table_upd.sort_values(by=['score'], ascending=False)

,AHA_adress,IRS_address,score
2922,"1905 AMERICAN WAY , KINGSPORT , TN , 37660","1905 AMERICAN WAY , KINGSPORT , TN , 37660",100
1528,"1350 WALTON WAY , AUGUSTA , GA , 30901","1350 WALTON WAY , AUGUSTA , GA , 30901",100
1537,"1601 WATSON BOULEVARD , WARNER ROBINS , GA , 3...","1601 WATSON BOULEVARD , WARNER ROBINS , GA , 3...",100
1542,"1601 WATSON BOULEVARD , WARNER ROBINS , GA , 3...","1601 WATSON BOULEVARD , WARNER ROBINS , GA , 3...",100
1544,"35 HOSPITAL ROAD , BLAIRSVILLE , GA , 30512","35 HOSPITAL ROAD , BLAIRSVILLE , GA , 30512",100
1549,"20555 VICTOR PARKWAY , LIVONIA , MI , 48152","20555 VICTOR PARKWAY , LIVONIA , MI , 48152",100
1554,"600 EAST DIXIE AVENUE , LEESBURG , FL , 34748","600 EAST DIXIE AVENUE , LEESBURG , FL , 34748",100
1555,"900 HOPE WAY , ALTAMONTE SPRINGS , FL , 32714","900 HOPE WAY , ALTAMONTE SPRINGS , FL , 32714",100
1556,"20555 VICTOR PARKWAY , LIVONIA , MI , 48152","20555 VICTOR PARKWAY , LIVONIA , MI , 48152",100
5175,"20555 VICTOR PARKWAY , LIVONIA , MI , 48152","20555 VICTOR PARKWAY , LIVONIA , MI , 48152",100


In [10]:
merge_table_upd['score'].value_counts()

-1      2746
 100     550
 90      130
 89       89
 96       72
 94       62
 88       62
 85       44
 81       33
 80       32
 83       32
 95       32
 91       27
 77       22
 82       21
 97       20
 87       17
 93       15
 92       11
 98       10
 76       10
 78        9
 79        7
 84        3
Name: score, dtype: int64

In [12]:
IRS2016.to_csv("IRS2016.csv")
aha2016.to_csv("aha2016.csv")
merge_table_upd.to_csv("merge_table_upd.to_csv")